# 주변 장소를 호출하는 함수입니다: Google 장소 API 및 고객 프로필 활용하기

이 노트북은 위치 기반 검색을 개선하기 위해 Google 장소 API와 사용자 지정 사용자 프로필을 통합하는 데 중점을 두고 있습니다. Google의 접근 방식은 Google Places API를 사용자 기본 설정과 함께 사용하여 보다 개인적이고 관련성 높은 위치 검색을 제공하는 것을 목표로 합니다. 이 사례에서는 Google Places API에 중점을 두었지만, 이와 유사한 방식으로 탐색하고 적용할 수 있는 다른 API가 많이 있다는 점에 유의하세요.

세 가지 주요 구성 요소의 적용에 대해 살펴보겠습니다:

- 고객 프로필: 이 모의 프로필은 장소 유형(예: 레스토랑, 공원, 박물관), 예산, 선호 평점 및 기타 특정 요구 사항에 대한 개별 선호도를 캡처합니다.

- Google 장소 API: 이 API는 주변 장소에 대한 실시간 데이터를 제공합니다. 주변 장소의 평점, 장소 유형, 비용 등 다양한 데이터 포인트를 고려합니다.

- 함수 호출: "배고파" 또는 "박물관에 가고 싶어"와 같은 명령어를 한 번만 입력하면 사용자 프로필 데이터와 Google 장소 API를 결합하여 적합한 장소를 식별하는 기능이 활성화됩니다.

이 노트북에서는 두 가지 주요 사용 사례를 소개합니다:

- 프로필 기반 추천: 사용자 프로필을 생성하고 개인의 선호도에 따라 장소를 추천하는 방법을 알아보세요.

- 함수 호출을 통한 API 통합: 함수 호출을 사용하여 다양한 장소의 실시간 데이터를 소싱하기 위해 Google 장소 API를 효과적으로 통합하고 호출하는 방법을 이해합니다.

이 시스템은 매우 다재다능하지만 사용자 선호도와 사용 가능한 장소 데이터에 따라 그 효과가 달라질 수 있다는 점에 유의하세요. 이 노트북의 목적상 고객 데이터는 가짜였으며 위치는 하드코딩되어 있습니다.

설정 ## 설정

Google 장소 API

Google Places API를 사용하려면 두 가지가 필요합니다:

- Google 계정: 아직 계정이 없는 경우 Google 계정을 만들어야 합니다.

- Google 지역정보 API 키: API 키는 사용 및 청구 목적으로 프로젝트와 관련된 요청을 인증하는 데 사용되는 고유 식별자입니다. API 키는 [Google 클라우드 콘솔](https://console.cloud.google.com/getting-started?authuser=1)에서 받을 수 있습니다.



Google Places API는 유료 서비스이며, 비용은 API 호출 횟수에 따라 부과됩니다. 예상치 못한 요금이 부과되지 않도록 사용량을 추적하세요.



요청 라이브러리도 필요하며, 다음 명령을 사용하여 다운로드할 수 있습니다:

```python
pip 설치 요청

In [1]:
import json
import openai
import os
import requests

이 코드 스니펫에서는 'user_id'를 받아 모의 사용자 프로필을 반환하는 함수 `fetch_customer_profile`을 정의하고 있습니다.

이 함수는 데이터베이스에서 사용자 데이터를 가져오는 API 호출을 시뮬레이션합니다. 이 데모에서는 하드코딩된 데이터를 사용합니다. 사용자 프로필에는 사용자의 위치(이 예에서는 금문교 좌표로 설정됨), 음식 및 활동 선호도, 앱 사용 지표, 최근 상호 작용 및 사용자 순위와 같은 다양한 세부 정보가 포함됩니다.

프로덕션 환경에서는 이 하드코딩된 데이터를 사용자 데이터베이스에 대한 실제 API 호출로 대체할 수 있습니다.


In [2]:
def fetch_customer_profile(user_id):
    # You can replace this with a real API call in the production code
    if user_id == "user1234":
        return {
            "name": "John Doe",
            "location": {
                "latitude": 37.7955,
                "longitude": -122.4026,
            },
            "preferences": {
                "food": ["Italian", "Sushi"],
                "activities": ["Hiking", "Reading"],
            },
            "behavioral_metrics": {
                "app_usage": {
                    "daily": 2,  # hours
                    "weekly": 14  # hours
                },
                "favourite_post_categories": ["Nature", "Food", "Books"],
                "active_time": "Evening",
            },
            "recent_searches": ["Italian restaurants nearby", "Book clubs"],
            "recent_interactions": ["Liked a post about 'Best Pizzas in New York'", "Commented on a post about 'Central Park Trails'"],
            "user_rank": "Gold",  # based on some internal ranking system
        }
    else:
        return None


## Google 플레이스 API에서 데이터 요청 및 처리하기

call_google_places_api 함수는 Google 장소 API에 정보를 요청하여 지정된 장소 유형과 선택적 음식 선호도에 따라 상위 두 곳의 장소 목록을 제공하는 역할을 합니다. 이 기능은 유료 서비스이므로 사용량을 관리하기 위해 상위 두 개의 결과로 제한했습니다. 그러나 요구 사항에 따라 원하는 만큼의 결과를 검색하도록 수정할 수 있습니다.

이 기능은 하드코딩된 위치(트랜스아메리카 피라미드의 좌표로 설정됨), Google API 키 및 특정 요청 매개변수로 구성됩니다. place_type에 따라 적절한 API 요청 URL을 공식화합니다. place_type이 레스토랑이고 food_preference가 지정되어 있는 경우 API 요청에 포함됩니다.

GET 요청을 전송한 후 함수는 응답 상태를 확인합니다. 성공하면 JSON 응답을 처리하고 get_place_details 함수를 사용하여 관련 세부 정보를 추출한 후 사람이 읽을 수 있는 형식으로 반환합니다. 요청이 실패하면 디버깅을 위해 오류를 출력합니다.

get_place_details 함수는 place_id가 주어진 장소에 대한 더 자세한 정보를 검색하는 데 사용됩니다. 이 함수는 Google 장소 정보 API에 GET 요청을 전송하고 요청이 성공하면 결과를 반환합니다. 요청이 실패하면 디버깅을 위해 오류를 출력합니다.

두 함수 모두 예외를 처리하고 문제가 발생하면 오류 메시지를 반환합니다.

In [3]:
def get_place_details(place_id, api_key):
    URL = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&key={api_key}"
    response = requests.get(URL)
    if response.status_code == 200:
        result = json.loads(response.content)["result"]
        return result
    else:
        print(f"Google Place Details API request failed with status code {response.status_code}")
        print(f"Response content: {response.content}")
        return None

In [4]:
def call_google_places_api(user_id, place_type, food_preference=None):
    try:
        # Fetch customer profile
        customer_profile = fetch_customer_profile(user_id)
        if customer_profile is None:
            return "I couldn't find your profile. Could you please verify your user ID?"

        # Get location from customer profile
        lat = customer_profile["location"]["latitude"]
        lng = customer_profile["location"]["longitude"]

        API_KEY = os.getenv('GOOGLE_PLACES_API_KEY')  # retrieve API key from environment variable
        LOCATION = f"{lat},{lng}"
        RADIUS = 500  # search within a radius of 500 meters
        TYPE = place_type

        # If the place_type is restaurant and food_preference is not None, include it in the API request
        if place_type == 'restaurant' and food_preference:
            URL = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={LOCATION}&radius={RADIUS}&type={TYPE}&keyword={food_preference}&key={API_KEY}"
        else:
            URL = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={LOCATION}&radius={RADIUS}&type={TYPE}&key={API_KEY}"

        response = requests.get(URL)
        if response.status_code == 200:
            results = json.loads(response.content)["results"]
            places = []
            for place in results[:2]:  # limit to top 2 results
                place_id = place.get("place_id")
                place_details = get_place_details(place_id, API_KEY)  # Get the details of the place
                
                place_name = place_details.get("name", "N/A")
                place_types = next((t for t in place_details.get("types", []) if t not in ["food", "point_of_interest"]), "N/A")  # Get the first type of the place, excluding "food" and "point_of_interest"
                place_rating = place_details.get("rating", "N/A")  # Get the rating of the place
                total_ratings = place_details.get("user_ratings_total", "N/A")  # Get the total number of ratings
                place_address = place_details.get("vicinity", "N/A")  # Get the vicinity of the place
                
                if ',' in place_address:  # If the address contains a comma
                    street_address = place_address.split(',')[0]  # Split by comma and keep only the first part
                else:
                    street_address = place_address
                
                # Prepare the output string for this place
                place_info = f"{place_name} is a {place_types} located at {street_address}. It has a rating of {place_rating} based on {total_ratings} user reviews."
                
                places.append(place_info)

            return places
        else:
            print(f"Google Places API request failed with status code {response.status_code}")
            print(f"Response content: {response.content}")  # print out the response content for debugging
            return []
    except Exception as e:
        print(f"Error during the Google Places API call: {e}")
        return []

## GPT-3.5-Turbo 및 Google Places API로 사용자별 추천 생성하기

'제공_사용자별_추천' 함수는 GPT-3.5-Turbo 및 Google Places API와 상호 작용하여 사용자의 선호도와 위치에 맞는 응답을 제공합니다.

먼저, 'user_id'를 사용하여 고객의 프로필을 가져옵니다. 프로필을 찾을 수 없으면 오류 메시지를 반환합니다.

유효한 프로필이 있으면 고객의 음식 선호도를 추출한 다음 OpenAI 모델과 상호 작용합니다. 초기 시스템 메시지를 제공하여 AI 모델의 역할, 사용자 기본 설정 및 Google Places API 기능 사용에 대한 컨텍스트를 제공합니다.

사용자 입력도 메시지로 모델에 전송되며, 필요에 따라 AI 모델이 호출할 수 있도록 `functions` 파라미터에 `call_google_places_api` 함수가 정의되어 있습니다.

마지막으로 모델의 응답을 처리합니다. 모델이 구글 플레이스 API에 함수를 호출하면 적절한 인수와 함께 함수가 실행되고 주변 장소의 이름이 반환됩니다. 해당 장소가 없거나 요청을 이해할 수 없는 경우 적절한 오류 메시지가 반환됩니다.


In [14]:
def provide_user_specific_recommendations(user_input, user_id):
    customer_profile = fetch_customer_profile(user_id)
    if customer_profile is None:
        return "I couldn't find your profile. Could you please verify your user ID?"
    
    customer_profile_str = json.dumps(customer_profile)

    food_preference = customer_profile.get('preferences', {}).get('food', [])[0] if customer_profile.get('preferences', {}).get('food') else None
  

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
    {
        "role": "system",
        "content": f"You are a sophisticated AI assistant, a specialist in user intent detection and interpretation. Your task is to perceive and respond to the user's needs, even when they're expressed in an indirect or direct manner. You excel in recognizing subtle cues: for example, if a user states they are 'hungry', you should assume they are seeking nearby dining options such as a restaurant or a cafe. If they indicate feeling 'tired', 'weary', or mention a long journey, interpret this as a request for accommodation options like hotels or guest houses. However, remember to navigate the fine line of interpretation and assumption: if a user's intent is unclear or can be interpreted in multiple ways, do not hesitate to politely ask for additional clarification. Make sure to tailor your responses to the user based on their preferences and past experiences which can be found here {customer_profile_str}"
    },
    {"role": "user", "content": user_input}
],
        temperature=0,
        functions=[
            {
                "name": "call_google_places_api",
                "description": "This function calls the Google Places API to find the top places of a specified type near a specific location. It can be used when a user expresses a need (e.g., feeling hungry or tired) or wants to find a certain type of place (e.g., restaurant or hotel).",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "place_type": {
                            "type": "string",
                            "description": "The type of place to search for."
                        }
                    }
                },
                "result": {
                    "type": "array",
                    "items": {
                        "type": "string"
                    }
                }
            }
        ],
    )

    if 'message' in response.choices[0] and 'function_call' in response.choices[0]['message']:
        function_call = response.choices[0]['message']['function_call']
        if function_call["name"] == "call_google_places_api":
            place_type = json.loads(function_call["arguments"])["place_type"]
            places = call_google_places_api(user_id, place_type, food_preference)
            if places:  # If the list of places is not empty
                return f"Here are some places you might be interested in: {' '.join(places)}"
            else:
                return "I couldn't find any places of interest nearby."

    return "I am sorry, but I could not understand your request."



## 사용자별 권장 사항 실행하기

함수가 실행되면 사용자의 프로필을 가져오고, AI 모델과 상호 작용하고, 모델의 응답을 처리하고, 필요한 경우 Google Places API를 호출하고, 궁극적으로 사용자의 선호도와 위치에 맞는 추천 목록을 반환합니다. 인쇄된 출력물은 이러한 개인화된 추천으로 구성됩니다.


In [15]:
user_id = "user1234"  
user_input = "I'm hungry"  
output = provide_user_specific_recommendations(user_input, user_id)
print(output)

Here are some places you might be interested in: Mona Lisa Restaurant is a restaurant located at 353 Columbus Avenue #3907. It has a rating of 4.3 based on 1784 user reviews. Tommaso's Ristorante Italiano is a restaurant located at 1042 Kearny Street. It has a rating of 4.5 based on 732 user reviews.
